In [1]:
using Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()
using Revise, Distributions, GLMakie, Dates, DataFrames, CSV, BenchmarkTools, DataFramesMeta
using TrypColonies
import Serialization as s

  Activating project at `c:\Users\Andreas\Dropbox\phd\TrypColonies\TrypColonies`

(process:23988): GLib-GIO-WARNING **: 15:17:59.455: Failed to open application manifest `C:\Windows\SystemApps\Microsoft.WindowsAppRuntime.CBS_8wekyb3d8bbwe\AppxManifest.xml' for package #85 (`Microsoft.WindowsAppRuntime.CBS_5001.373.1736.100_x64__8wekyb3d8bbwe'): error code 0x2


### Change folder to data director

In [5]:
data_path = find_data_path(local_data = false)

"C:\\Users\\Andreas\\Desktop\\simulation_data_julia\\"

In [6]:
data_base = s.deserialize("data_base\\df.jls") 
data_base = @subset(data_base, :location .== data_path)   
first(data_base, 5) 

Row,radius_tanget,boundary_conditions,agent_number,noise_dis,sweep_parameter,run_time,unix_date,path,location,geometry,parameter_increment,storage,noise_strength,velocity_variance,iterations,parameter_steps,velocity_dis,size,velocity,radius_collision,date,interaction_radius,grid_strength,start_config,path_tracing,timesteps,boundary_moveable,sliding_boundary
,Int64,String,Int64,UnionAll,String,Float64,Float64,String,String,String,Float64,Float64,Float64,Float64,Int64,Int64,UnionAll,Tuple…,Int64,Int64,DateTime,Int64,Int64,String,Bool,Int64,Bool,Bool
1,15,periodic,1000,Normal,noise_strength,146.859,1.72295e9,C:\\Users\\Andreas\\Desktop\\simulation_data_julia\\Classic_Vicsek_1000_time_seed_4380,C:\\Users\\Andreas\\Desktop\\simulation_data_julia\\,no_walls,0.1,1.34286,0.0,3.0,2,20,Normal,"(500, 500)",5,5,2024-08-06T14:28:48.903,7,20,random,false,1000,true,false
2,15,periodic,1000,Normal,noise_strength,163.425,1.72296e9,C:\\Users\\Andreas\\Desktop\\simulation_data_julia\\moveable_boundary_1000_time_seed_5508,C:\\Users\\Andreas\\Desktop\\simulation_data_julia\\,circle_small,0.1,1.47315,0.0,3.0,2,20,Normal,"(500, 500)",5,5,2024-08-06T14:40:50.477,7,20,random,false,1000,true,false
3,15,periodic,1000,Normal,noise_strength,172.533,1.72296e9,C:\\Users\\Andreas\\Desktop\\simulation_data_julia\\moveable_boundary_1000_time_seed_3540,C:\\Users\\Andreas\\Desktop\\simulation_data_julia\\,circle_small,0.1,1.91678,0.0,3.0,2,20,Normal,"(500, 500)",5,5,2024-08-06T17:01:53.012,7,20,random,true,1000,true,false


In [7]:
sort!(data_base, :unix_date)

Row,radius_tanget,boundary_conditions,agent_number,noise_dis,sweep_parameter,run_time,unix_date,path,location,geometry,parameter_increment,storage,noise_strength,velocity_variance,iterations,parameter_steps,velocity_dis,size,velocity,radius_collision,date,interaction_radius,grid_strength,start_config,path_tracing,timesteps,boundary_moveable,sliding_boundary
,Int64,String,Int64,UnionAll,String,Float64,Float64,String,String,String,Float64,Float64,Float64,Float64,Int64,Int64,UnionAll,Tuple…,Int64,Int64,DateTime,Int64,Int64,String,Bool,Int64,Bool,Bool
1,15,periodic,1000,Normal,noise_strength,146.859,1.72295e9,C:\\Users\\Andreas\\Desktop\\simulation_data_julia\\Classic_Vicsek_1000_time_seed_4380,C:\\Users\\Andreas\\Desktop\\simulation_data_julia\\,no_walls,0.1,1.34286,0.0,3.0,2,20,Normal,"(500, 500)",5,5,2024-08-06T14:28:48.903,7,20,random,false,1000,true,false
2,15,periodic,1000,Normal,noise_strength,163.425,1.72296e9,C:\\Users\\Andreas\\Desktop\\simulation_data_julia\\moveable_boundary_1000_time_seed_5508,C:\\Users\\Andreas\\Desktop\\simulation_data_julia\\,circle_small,0.1,1.47315,0.0,3.0,2,20,Normal,"(500, 500)",5,5,2024-08-06T14:40:50.477,7,20,random,false,1000,true,false
3,15,periodic,1000,Normal,noise_strength,172.533,1.72296e9,C:\\Users\\Andreas\\Desktop\\simulation_data_julia\\moveable_boundary_1000_time_seed_3540,C:\\Users\\Andreas\\Desktop\\simulation_data_julia\\,circle_small,0.1,1.91678,0.0,3.0,2,20,Normal,"(500, 500)",5,5,2024-08-06T17:01:53.012,7,20,random,true,1000,true,false


In [8]:
data_used = data_base[end,:]

Row,radius_tanget,boundary_conditions,agent_number,noise_dis,sweep_parameter,run_time,unix_date,path,location,geometry,parameter_increment,storage,noise_strength,velocity_variance,iterations,parameter_steps,velocity_dis,size,velocity,radius_collision,date,interaction_radius,grid_strength,start_config,path_tracing,timesteps,boundary_moveable,sliding_boundary
,Int64,String,Int64,UnionAll,String,Float64,Float64,String,String,String,Float64,Float64,Float64,Float64,Int64,Int64,UnionAll,Tuple…,Int64,Int64,DateTime,Int64,Int64,String,Bool,Int64,Bool,Bool
3,15,periodic,1000,Normal,noise_strength,172.533,1.72296e9,C:\\Users\\Andreas\\Desktop\\simulation_data_julia\\moveable_boundary_1000_time_seed_3540,C:\\Users\\Andreas\\Desktop\\simulation_data_julia\\,circle_small,0.1,1.91678,0.0,3.0,2,20,Normal,"(500, 500)",5,5,2024-08-06T17:01:53.012,7,20,random,true,1000,true,false


### Chose simulation run 

In [25]:
iteration_number = 1
parameter_step_number = 20
sim_run_number = (parameter_step_number-1)*(data_used.iterations)+iteration_number;
para_dict = make_para_dict_list(Dict(names(data_used) .=> values(data_used)))[sim_run_number]
para = create_para_struct(para_dict);

In [26]:
plot_para = plot_parameters(
    framerate        = 20, 
    theme            = theme_black(),
    fontsize         = 22,
    refresh_delay    = 0.0001,
    res              = (1000,1600),
    cor_func         = order_parameter,
    arrow_tip_size   = 10,
    arrow_tail_length= 15
    );
makie_config!(plot_para)

false

In [27]:
Full_Agent_list, grid_vec_sparse = s.deserialize(data_used.path*"\\"*readdir(data_used.path)[sim_run_number]);

Grid = Matrix(grid_vec_sparse[1])

f = Figure(size = plot_para.res)
display(f)
ax1,fig_obs = ini_animation(Full_Agent_list,para,Grid,f,plot_para);

In [28]:
fig_obs.Time[] = 1
for i in 2:data_used.timesteps
    animation_step!(ax1,Full_Agent_list,grid_vec_sparse, Grid,fig_obs,plot_para,para)
    isopen(f.scene) || break 
end

In [15]:
Grid = Matrix(grid_vec_sparse[1])
f = Figure(size = plot_para.res)
ax1,fig_obs = ini_animation(Full_Agent_list,para,Grid,f,plot_para);

fig_obs.Time[] = 1
record(f, "vids/test_vid_from_data.mp4",  2:data_used.timesteps, framerate = plot_para.framerate) do i
    animation_step!(ax1,Full_Agent_list,grid_vec_sparse, Grid,fig_obs,plot_para,para)
end

"vids/test_vid_from_data.mp4"